In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

## 사전 데이터 가공(LDA 추가 전)

In [2]:
df = pd.read_excel('E:/data20200301.xlsx')

In [3]:
f_date = [x for x in list(df.columns) if '2020' in x]

In [4]:
df2 = pd.DataFrame(df, columns=['notice_num','nick','mail','phone_fin',
                                'date','hour','minute','day',
                                'price','safe_phone',
                                'title','text',
                                'img_count','img_source',
                                ].extend(f_date))

In [5]:
date = list(df2['date'])

In [6]:
#df2['year'] = pd.Series([x.split('.')[0] for x in date])
#df2['month']= pd.Series([x.split('.')[1] for x in date])
#df2['days'] = pd.Series([x.split('.')[2] for x in date])
df2['date'] = pd.Series([datetime(int(x.split('.')[0]),int(x.split('.')[1]),int(x.split('.')[2])) for x in date])

In [7]:
df3 = df2.loc[:,f_date]

In [8]:
df3['total'] = 0
df3['s_search'] = 0
df3['r_search1'] = 0
df3['r_search2'] = 0
df3['r_search3'] = 0

In [9]:
for i in range(len(df3)):
    v = list(df3.iloc[i,0:len(f_date)])
    df3['total'][i] = v
    v_ = v.copy()
    v_.reverse()
    df3['s_search'][i] = datetime(2020,2,2) + relativedelta(days=len(v), months=-3)
    #df3['s_search'][i] = datetime(2020,2,2) + relativedelta(days=len(v) - v_.index('없음'),months=-3)
    df3['r_search1'][i] = v[-1]
    df3['r_search2'][i] = v[-2]
    df3['r_search3'][i] = v[-3]

<ipython-input-9-18eaeede084c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['total'][i] = v
C:\Users\dong\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-9-18eaeede084c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['s_search'][i] = datetime(2020,2,2) + relativedelta(days=len(v), months=-3)
<ipython-input-9-18

In [10]:
total = list(df3['total'])
s_search = list(df3['s_search'])
r_search1 = list(df3['r_search1'])
r_search2 = list(df3['r_search2'])
r_search3 = list(df3['r_search3'])

In [11]:
df2['Fraud'] = pd.Series([x for x in total])
df2['s_search'] = pd.Series([x for x in s_search])
df2['r_search1'] = pd.Series([x for x in r_search1])
df2['r_search2'] = pd.Series([x for x in r_search2])
df2['r_search3'] = pd.Series([x for x in r_search3])

In [12]:
df4 = df2.drop(f_date,axis = 1)

In [13]:
df4['Final'] = 0

In [14]:
df4.loc[(df4['date'] >= df4['s_search']) & (df4['r_search1'] == "Y") & (df4['r_search2'] == "Y") & (df4['r_search3'] == "Y"),['Final']] = "Y"

In [15]:
df4.loc[df4['Final'] == 0 ,['Final']] = "N"

In [16]:
df4.loc[df4['Final'] == 0 ,['Final']]

,Final


## LDA

In [17]:
import re
from pandas import Series
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
scaler = RobustScaler()
#scaler = StandardScaler()
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
%matplotlib inline

import konlpy
from konlpy.tag import Okt, Komoran
okt = Okt()
kmr = Komoran(userdic='/tmp/dic.txt')
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import warnings
from gensim.models.coherencemodel import CoherenceModel
## import pyLDAvis.gensim as gensimvis
## import pyLDAvis

from gensim.models import TfidfModel

C:\Users\dong\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [18]:
raw = df4

In [19]:
df = df4

def deEmojify(inputString):
    return str(inputString.encode('CP949', 'ignore').decode('CP949'))

## tokenizing
def preprocess(t):
    #token = okt.morphs(str(t).strip().replace('  ',''), stem = True)
    #prep = [word for word in token if not word in stop_words]
    
    #token = okt.pos(str(t), norm=True, stem = True)
    try:
        token = kmr.pos(deEmojify(" ".join(str(t).splitless())))
    except:
        token = okt.pos(str(t))
        
    prep = [word for word, pos in token if pos in p]
    
    return prep

In [20]:
# p = ['Noun','Verb','Adjective','Adverb'] #okt

#p = ['NNG','NNP','NNB','NR','NP','VV','VA','MAG','MAJ',
#     'Noun','Verb','Adjective','Adverb'] #kmr

n = ['Noun']

v = ['Verb']

aj = ['Adjective']

av = ['Adverb']

In [21]:
## tokenizing
def preprocess(t):
    token = okt.pos(str(t), norm=True, stem = True)
    ## len(token)
    if len(token) == 0:
        pp_n.append(0)
        pp_v.append(0)
        pp_aj.append(0)
        pp_av.append(0)
    else:
        pp_n.append(len([word for word, pos in token if pos in n])/len(token))
        pp_v.append(len([word for word, pos in token if pos in n])/len(token))
        pp_aj.append(len([word for word, pos in token if pos in n])/len(token))
        pp_av.append(len([word for word, pos in token if pos in n])/len(token))
            
    p_n.append([word for word, pos in token if pos in n])
    p_v.append([word for word, pos in token if pos in v])
    p_aj.append([word for word, pos in token if pos in aj])
    p_av.append([word for word, pos in token if pos in av])
           
    pass

In [22]:
p_n = []
p_v = []
p_aj = []
p_av = []
pp_n = []
pp_v = []
pp_aj = []
pp_av = []

print(len(raw['text']))
raw['text'].apply(lambda x: preprocess(x))
print(len(p_n),len(pp_n))
print(len(p_v),len(pp_v))
print(len(p_aj),len(pp_aj))
print(len(p_av),len(pp_av))

145536
145536 145536
145536 145536
145536 145536
145536 145536


tag = p_aj
dictionary = Dictionary(tag)
pre_corpus = [dictionary.doc2bow(text) for text in tag]
model = TfidfModel(pre_corpus)
corpus = Series(pre_corpus).apply(lambda x: model[x])

for n in [2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
    lda = LdaModel(corpus, id2word=dictionary, num_topics=n, alpha = 0.1, eta = 50/n, eval_every = -1, passes=10, random_state = 1)
    print('\n',n, '개의 주제로 나눴을 경우','\nPerplexity: ', lda.log_perplexity(corpus),)
    coherence_model_lda = CoherenceModel(model=lda, texts=tag, dictionary=dictionary, coherence='c_v')
    print('Coherence Score: ', coherence_model_lda.get_coherence())

n = 2
lda = LdaModel(corpus, id2word=dictionary, num_topics=n, alpha = 0.1, eta = 50/n, eval_every = -1, passes=10, random_state = 1)

lda_topic = lda.print_topics(num_topics=n, num_words=10)
f = open("lda.txt", 'w', -1, "utf-8")
count = 1
for i in lda_topic:
    f.write('---- {}번째 주제 -----\n'.format(count))
    print('\n---- {}번째 주제 -----'.format(count))
    count += 1
    b = i[1].split('+')
    for c in b:
        [value, word] = c.split('*')
        f.write('{}:\t{}\n'.format(word.split('"')[1], value.split("'")[0].strip()))
        print('{}:\t{}'.format(word.split('"')[1], value.split("'")[0].strip()))
f.close()

In [23]:
n = 2
for name, pre_text in zip(['n','v','aj','av'], [p_n,p_v,p_aj,p_av]):
    dictionary = Dictionary(pre_text)
    #dictionary.filter_extremes(no_below=2000, no_above=0.5) #TF-IDF로 대체? or같이 사용?
    pre_corpus = [dictionary.doc2bow(text) for text in pre_text]
    model = TfidfModel(pre_corpus)

    corpus = Series(pre_corpus).apply(lambda x: model[x])
    print('\n' + name)
    globals()[f'{name}_lda'] = LdaModel(corpus, id2word=dictionary, num_topics=n, alpha = 0.1, eta = 1.5, eval_every = -1, passes=10, random_state = 1)
    globals()[f'lda_topic_{name}'] = globals()[f'{name}_lda'].print_topics(num_topics=n, num_words=10)
    f = open("lda.txt", 'w', -1, "utf-8")
    count = 1
    for i in globals()[f'lda_topic_{name}']:
        f.write('---- {}번째 주제 -----\n'.format(count))
        print('\n---- {}번째 주제 -----'.format(count))
        count += 1
        b = i[1].split('+')
        for c in b:
            [value, word] = c.split('*')
            f.write('{}:\t{}\n'.format(word.split('"')[1], value.split("'")[0].strip()))
            print('{}:\t{}'.format(word.split('"')[1], value.split("'")[0].strip()))
    f.close()
    globals()[f'topic_probability_mat_{name}'] = globals()[f'{name}_lda'][corpus]
    globals()[f'sparse_{name}'] = pd.DataFrame(data = globals()[f'topic_probability_mat_{name}'], columns = range(2))
    globals()[f'sparse_{name}'].loc[:,:]=0
    for i in range(len(globals()[f'topic_probability_mat_{name}'])):
        for (j,x) in globals()[f'topic_probability_mat_{name}'][i]:
            globals()[f'sparse_{name}'].loc[i,j] = x
    globals()[f'sparse_{name}'].columns = [name+'_1',name+'_2']


n

---- 1번째 주제 -----
상품:	0.005
사이즈:	0.005
상태:	0.005
용감:	0.005
문자:	0.004
택포:	0.004
문의:	0.004
가죽:	0.003
구매:	0.003
판매:	0.003

---- 2번째 주제 -----
제품:	0.006
판매:	0.005
연락:	0.004
구매:	0.004
직거래:	0.004
가격:	0.004
택배:	0.004
거래:	0.004
지갑:	0.004
구입:	0.004

v

---- 1번째 주제 -----
하다:	0.128
줄다:	0.118
파다:	0.050
드리다:	0.050
되다:	0.044
않다:	0.036
보내다:	0.033
받다:	0.031
바라다:	0.026
보다:	0.026

---- 2번째 주제 -----
하다:	0.040
입다:	0.033
되다:	0.023
적다:	0.022
줄다:	0.021
않다:	0.020
드리다:	0.019
되어다:	0.019
들다:	0.017
나오다:	0.016

aj

---- 1번째 주제 -----
있다:	0.047
이다:	0.044
좋다:	0.038
예쁘다:	0.037
없다:	0.028
가능하다:	0.027
이쁘다:	0.023
아니다:	0.022
부탁드리다:	0.018
고급스럽다:	0.018

---- 2번째 주제 -----
이다:	0.173
좋다:	0.098
있다:	0.097
가능하다:	0.086
없다:	0.071
부탁드리다:	0.063
깨끗하다:	0.037
많다:	0.034
원하다:	0.029
신중하다:	0.028

av

---- 1번째 주제 -----
다:	0.207
너무:	0.198
엄청:	0.052
훨씬:	0.032
함께:	0.032
오래:	0.025
물론:	0.025
혹은:	0.024
풍:	0.021
그래서:	0.021

---- 2번째 주제 -----
없이:	0.210
많이:	0.184
같이:	0.124
또는:	0.102
함께:	0.064
딱:	0.053
따로:	0.049
다:	0.040
아직:	0.024
히:	0.013


In [24]:
sparse_n

,n_1,n_2
0,0.026923,0.973077
1,0.0174832,0.982517
2,0.0224072,0.977593
3,0.0262071,0.973793
4,0.985655,0.0143451
...,...,...
145531,0.972207,0.0277927
145532,0.0260272,0.973973
145533,0.985024,0.0149759
145534,0.0181809,0.981819


In [25]:
sparse_n.columns = ["n_1","n_2"]
sparse_v.columns = ["v_1","v_2"]
sparse_aj.columns = ["aj_1","aj_2"]
sparse_av.columns = ["av_1","av_2"]
r_n = pd.DataFrame(data = pp_n, columns = ['r_n'])
r_v = pd.DataFrame(data = pp_v, columns = ['r_v'])
r_aj = pd.DataFrame(data = pp_aj, columns = ['r_aj'])
r_av = pd.DataFrame(data = pp_av, columns = ['r_av'])

## 마지막 전처리

def spch(Daf):
    if len(str(df['text'][i])) == 0:
        return 0
    else:
        return len(re.sub(r'\s','',re.sub(r'\w','',str(df['text'])))) / len(str(df['text']))
    
def whsp(Daf):
    if len(str(df['text'][i])) == 0:
        return 0
    else:
        return len(re.sub(r'\S','',re.sub(r'\w','',str(df['text'])))) / len(str(df['text']))

def num(Daf):
    if len(str(df['text'][i])) == 0:
        return 0
    else:
        return len(re.sub(r'\s','',re.sub(r'\D','',str(df['text'])))) / len(str(df['text']))

def zero(Daf):
    return int(str(df['price']).count('0')) / len(str(df['price']))

def mail_hide(Daf):
    if '*' in df4['mail']:
        return 1
    else :
        return 0
    

df['ratio_spch'] = df.apply(spch, axis=1)
df['ratio_whsp'] = df.apply(whsp, axis=1)
df['ratio_num'] = df.apply(num, axis=1)
df['price_zero'] = df.apply(zero, axis=1)
df['mail_hide'] = df.apply(mail_hide, axis=1)

### 특징

In [26]:
df['mail_hide'] = 0
df['len_text'] = 0
df['ratio_spch'] = 0
df['ratio_whsp'] = 0
df['ratio_num'] = 0
df['price_zero'] = 0

for i in range(len(df['Unnamed: 0'])):
    df.loc[i,'price_zero'] = float(str(df['price'][i]).count('0') / len(str(df['price'][i])))
    if '*' in df4['mail'][i]:
        df['mail_hide'][i] = 1
    df.loc[i,'len_text'] = len(str(df['text'][i]))
    if len(str(df['text'][i])) != 0:
        df.loc[i,'ratio_spch'] = len(re.sub(r'\s','',re.sub(r'\w','',str(df['text'][i])))) / len(str(df['text'][i]))
        df.loc[i,'ratio_whsp'] = len(re.sub(r'\S','',str(df['text'][i]))) / len(str(df['text'][i]))
        df.loc[i,'ratio_num'] = len(re.sub(r'\D','',str(df['text'][i]))) / len(str(df['text'][i]))

<ipython-input-26-1007c5f81245>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mail_hide'][i] = 1


In [27]:
col_c = ['img_count','len_text','ratio_spch','ratio_whsp','ratio_num','price_zero'] ## 연속형 변수
col_n = ['day','grade','hour','mail_hide','safe_phone'] ## 명목형 변수

In [28]:
df.loc[df['Final']== 'Y', ['Final']] = 1
df.loc[df['Final']== 'N', ['Final']] = 0
Y = df['Final']
X = pd.DataFrame(df,columns = col_c + col_n)

In [29]:
A = pd.get_dummies(X.day, prefix='day')
B = pd.get_dummies(X.grade, prefix='grade')
C = pd.get_dummies(X.hour, prefix='hour')
D = pd.get_dummies(X.mail_hide, prefix='hide')
E = pd.get_dummies(X.safe_phone, prefix='safe')

pro_fin = pd.concat([X.drop(col_n,axis=1),A,B,C,D,E,sparse_n,sparse_v,sparse_aj,sparse_av,r_n,r_v,r_aj,r_av], axis=1)

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(pro_fin, Y, random_state=0)#, stratify=Y)

In [31]:
X_train.loc[:,col_c] = scaler.fit_transform(X_train.loc[:,col_c])
X_test.loc[:,col_c] = scaler.transform(X_test.loc[:,col_c])

C:\Users\dong\Anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
C:\Users\dong\Anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


### 오버샘플링

In [32]:
from imblearn.over_sampling import SMOTENC

In [33]:
print("After OverSampling, counts of label '0': {}".format(sum(Y_train==0)))
print("After OverSampling, counts of label '1': {}".format(sum(Y_train==1)))

After OverSampling, counts of label '0': 108530
After OverSampling, counts of label '1': 622


In [34]:
dic = {}
for n, c in enumerate(list(X_train.columns)):
    dic[c] = n

In [35]:
cf = [list(i.columns) for i in [A,B,C,D,E]]
cf = sum(cf,[])

In [36]:
smote_nc = SMOTENC(categorical_features=[dic[x] for x in cf], sampling_strategy=0.3, random_state=0)
Y_train_r = pd.Series(Y_train.astype('float'))
X_res, Y_res = smote_nc.fit_resample(X_train, Y_train_r)

In [37]:
print("After OverSampling, counts of label '0': {}".format(sum(Y_res==0)))
print("After OverSampling, counts of label '1': {}".format(sum(Y_res==1)))

After OverSampling, counts of label '0': 108530
After OverSampling, counts of label '1': 32559


In [38]:
colu = list(pro_fin.columns)

In [39]:
X_res.to_excel('train_X.xlsx')
X_test.to_excel('test_X.xlsx')
Y_res.to_excel('train_Y.xlsx')
Y_test.to_excel('test_Y.xlsx')